In [13]:
import tensorflow as tf
import numpy as np
import json
import pandas as pd
import random

In [14]:
with open("E:/MLCourse/foxBot/foxPython/End2EndModel/data/intentsVNver1.json",encoding="utf-8") as file:
    data=json.load(file)
patternlist=[]
labels=[]
for dt in data["intents"]:
    for pattern in dt["patterns"]:
        patternlist.append(pattern)
        labels.append(dt["tag"])
labels
labels=np.array(labels)
labelsOH=pd.get_dummies(labels)
labellist=list(labelsOH)

with open("E:/MLCourse/foxBot/foxPython/End2EndModel/data/bi_grams.txt",encoding="utf-8") as f:
    bi_gramtxt=f.read()
bi_gram_list=[]
for each in bi_gramtxt.split(","):
    each=each[:-1]
    each=each[2:]
    each=each.replace(" ","_")
    bi_gram_list.append(each)

with open("E:/MLCourse/foxBot/foxPython/End2EndModel/data/vietnamese-stopwords.txt",encoding="utf-8") as f:
    stopword_txt=f.readlines()
    
stopword_list=[]
for each in stopword_txt:
    stopword_list.append(each[:-1])

with open("E:/MLCourse/foxBot/foxPython/End2EndModel/data/posdata.json","r",encoding="utf-8") as f:
    mpos=json.load(f)

responses_dict={}
for i in data["intents"]:
    responses_dict[i["tag"]]=i["responses"]

with open("E:/MLCourse/foxBot/foxPython/End2EndModel/data/knowledge.json","r",encoding="utf-8") as f:
    knowledge=json.load(f)

In [15]:
list(knowledge.values())[0]["pizza"]

{'favorite': 'yes', 'info': ['ngon', 'rất ngon'], 'xuất xứ': 'Ý'}

In [16]:
list(knowledge.values())[0]


{'pizza': {'favorite': 'yes', 'info': ['ngon', 'rất ngon'], 'xuất xứ': 'Ý'},
 'phở': {'favorite': 'yes',
  'info': ['ngon', 'ngon hơn nữa'],
  'xuất xứ': 'Việt Nam'},
 'đậu xanh': {'favorite': 'no', 'info': ['không ngon']}}

In [17]:
class brain_exe():
    def __init__(self,knowledge):
        self.mem={}
        self.dialogue_count=0
        self.length_limit=20
        self.knowledge_base=knowledge

    def pos_filter(self,pos_rs,pos_name,stopword=["bạn","tớ","cậu","tí","này","lát","chút","nhé","nha","mình"]):
        pos_filtered=[]
        for i in pos_rs:
            if (i[1]==pos_name and stopword.count(i[0])==0):
                pos_filtered.append(i)
        return pos_filtered

    def reset(self):
        self.mem={}
    
    def add(self,intent,sentence_entities):
        self.mem[self.dialogue_count]=[intent,sentence_entities]
        self.dialogue_count=self.dialogue_count+1
        if self.dialogue_count==self.length_limit:
            self.mem={}

    def lookup(self):
        return self.mem

    def knowledge_lookup(self,entities):
        if len(entities)!=0:
            verb_list=self.pos_filter(entities,"verb")
            noun_list=self.pos_filter(entities,"noun")
            result=[]
            for i in noun_list:
                i[0]=i[0].replace("_"," ")
                for each in list(knowledge.values()):
                    for j in each.keys():
                        if i[0]==j:
                            result.append([j,each[j]])
            return result
        else:
            return False

In [18]:
class FTokenizer():
    def __init__(self,bi_gram_list):
        self.bi_gram_list=bi_gram_list

    def bi_gram_checker(self,bi_gram_word):
        for i in self.bi_gram_list:
            if (bi_gram_word==i):
                return True
        return False
    
    def fit_on_text(self,data):
        word_to_index={}
        index_to_word={}
        word_to_index["<PAD>"]=0
        word_to_index["<OOV>"]=1
        index_to_word[0]="<PAD>"
        index_to_word[1]="<OOV>"
        myContinue=False
        tokens=[]
        for eachseq in data:
            listtokens=eachseq.lower().split(" ")
            listtokens.append("")
            for i in range(0,len(listtokens)-1):
                if (myContinue):
                    myContinue=False
                    continue
                if (listtokens[i+1]!=""):
                    pairtext=listtokens[i]+"_"+listtokens[i+1]
                    if (self.bi_gram_checker(pairtext)):
                        myContinue=True
                        if (tokens.count(pairtext)==0):
                            tokens.append(pairtext)
                if (myContinue==False):  
                    if (tokens.count(listtokens[i])==0):
                        tokens.append(listtokens[i])
        for index,word in enumerate(tokens):
            word_to_index[word]=index+2
            index_to_word[index+2]=word
        self.word_to_index=word_to_index
        self.index_to_word=index_to_word

    def fit_on_text2(self,data):
        word_to_index={}
        index_to_word={}
        word_to_index["<PAD>"]=0
        word_to_index["<OOV>"]=1
        index_to_word[0]="<PAD>"
        index_to_word[1]="<OOV>"
        myContinue=False
        tokens=[]
        for eachseq in data:
            listtokens=eachseq.lower().split(" ")
            listtokens.append("")
            for i in range(0,len(listtokens)-1):
                if (listtokens[i+1]!=""):
                    pairtext=listtokens[i]+"_"+listtokens[i+1]
                    if (self.bi_gram_checker(pairtext)):
                        if (tokens.count(pairtext)==0):
                            tokens.append(pairtext)
                if (tokens.count(listtokens[i])==0 and listtokens[i]!=""):
                    tokens.append(listtokens[i])
        for index,word in enumerate(tokens):
            word_to_index[word]=index+2
            index_to_word[index+2]=word
        self.word_to_index=word_to_index
        self.index_to_word=index_to_word
    
    def get_word_to_index(self):
        return self.word_to_index
    
    def get_index_to_word(self):
        return self.index_to_word

    def get_max_length(self,data):
        maxsen=0
        for i in data:
            if maxsen<len(i):
                maxsen=len(i)
        return maxsen

    def text_to_sequence(self,textdata):
        d_continue=False
        output=[]
        for text in textdata:
            text=text.lower().split(" ")
            text.append("")
            sth=[]
            d_continue=False
            for i in range(0,len(text)-1):
                if d_continue==True:
                    d_continue=False
                    continue
                if (text[i+1]!=""):
                    pairtext=text[i]+"_"+text[i+1]
                    for j in self.word_to_index:
                        if pairtext.lower()==j:
                            sth.append(self.word_to_index[j])
                            d_continue=True
                            break
                if d_continue==False:
                    for count,j in enumerate(self.word_to_index):
                        if text[i].lower()==j:
                            sth.append(self.word_to_index[j])
                            break
                        if count==len(self.word_to_index)-1:
                            sth.append(1)
            output.append(sth)
        return output
    
    def intseq_to_index(self,sequence):
        output=[]
        for i in sequence:
            output.append(self.index_to_word[i])
        return output

    def pad_sequence(self,datasequence,maxlen):
        for i in datasequence:
            while len(i)!=maxlen:
                i.insert(0,0)
        return datasequence

    def bi_gram_ize(self,text):
        listtokens=text.lower().split(" ")
        listtokens.append("")
        myContinue=False
        tokens=[]   
        for i in range(0,len(listtokens)-1):
            if (myContinue):
                myContinue=False
                continue
            if (listtokens[i+1]!=""):
                pairtext=listtokens[i]+"_"+listtokens[i+1]
                if (self.bi_gram_checker(pairtext)):
                    myContinue=True
                    tokens.append(pairtext)
            if (myContinue==False):  
                tokens.append(listtokens[i])
        return tokens


In [19]:
myTokenizer=FTokenizer(bi_gram_list)
myTokenizer.fit_on_text2(patternlist)
pattern_tokenized2=myTokenizer.text_to_sequence(patternlist)
pattern_preprocessed2=tf.keras.preprocessing.sequence.pad_sequences(pattern_tokenized2)
mlen2=pattern_preprocessed2.shape[1]

posmesTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000,filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n',oov_token="<UNK>")
posmesTokenizer.fit_on_texts(mpos["messages"])
mes_Tokenized=posmesTokenizer.texts_to_sequences(mpos["messages"])
finalmes=tf.keras.preprocessing.sequence.pad_sequences(mes_Tokenized)
poslen=finalmes.shape[1]

postagTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000)
postagTokenizer.fit_on_texts(mpos["pos"])
pos_tokenized=postagTokenizer.texts_to_sequences(mpos["pos"])
finalpos=tf.keras.preprocessing.sequence.pad_sequences(pos_tokenized)
postagTokenizer.index_word[0]="<PAD>"

In [20]:
intentdict=myTokenizer.word_to_index
posmesdict=posmesTokenizer.word_index
postagdict=postagTokenizer.word_index
this_bi_gram_lict=bi_gram_list
posmesdict["<PAD>"]=0
postagdict["<PAD>"]=0
with open("E:/MLCourse/TF2/Notebook/tutorialNotebook/data/word_dictionary/intentDict.json","w",encoding="utf-8") as f:
    json.dump(intentdict,f)
with open("E:/MLCourse/TF2/Notebook/tutorialNotebook/data/word_dictionary/posmesDict.json","w",encoding="utf-8") as f:
    json.dump(posmesdict,f)
with open("E:/MLCourse/TF2/Notebook/tutorialNotebook/data/word_dictionary/postagDict.json","w",encoding="utf-8") as f:
    json.dump(postagdict,f)
with open("E:/MLCourse/TF2/Notebook/tutorialNotebook/data/word_dictionary/this_bg_list.json","w",encoding="utf-8") as f:
    json.dump(this_bi_gram_lict,f)  

In [21]:
class BotBackendLite():
    def __init__(self,posdata,intentdata,bi_gram_list,labellist):
        self.posdata=posdata
        self.intent_data=intentdata
        self.bi_gram_list=bi_gram_list
        self.labellist=np.array(labellist+["NULL"])
        self.response_dictionary=responses_dict
        self.response_dictionary["NULL"]=["Mình không hiểu lắm","Là sao vậy ? Mình không hiểu cậu lắm","Là gì thế ?"]

        self.interpreter=tf.lite.Interpreter("E:/MLCourse/foxBot/foxPython/End2EndModel/model/tfliteModel/model3.tflite")
        self.interpreter.allocate_tensors()
        self.input_details=self.interpreter.get_input_details()
        self.output_details=self.interpreter.get_output_details()
        self.input_shape1=self.input_details[0]["shape"]
        self.input_shape2=self.input_details[1]["shape"]
        self.brain=brain_exe(knowledge)

    def preprocess(self,inp):
        punc_list=[",",".","'","[","]","!","?"]
        for i in punc_list:
            inp=inp.replace(i,"")
        inp_tokenized=myTokenizer.text_to_sequence([inp])
        inp_preprocessed=tf.keras.preprocessing.sequence.pad_sequences(inp_tokenized,maxlen=mlen2)

        inp_gramized=myTokenizer.bi_gram_ize(inp)
        inp_tokenized=posmesTokenizer.texts_to_sequences([inp_gramized])
        final_inp=tf.keras.preprocessing.sequence.pad_sequences(inp_tokenized,maxlen=finalmes.shape[1])
        return inp_preprocessed,final_inp

    def posAnalyze(self,inp,posdata):
        pos=[]
        for i in posdata[0]:
            if postagTokenizer.index_word[np.argmax(i)]!="<PAD>":
                pos.append(postagTokenizer.index_word[np.argmax(i)])
        l=[]
        for i in range(0,len(pos)):
            if pos[i]!="<PAD>":
                l.append([inp[i],pos[i]])
        return l

    def pos_filter(self,pos_rs,pos_name,stopword=["bạn","tớ","cậu","tí","này","lát","chút","nhé","nha","mình"]):
        pos_filtered=[]
        for i in pos_rs:
            if (i[1]==pos_name and stopword.count(i[0])==0):
                pos_filtered.append(i)
        return pos_filtered

    def accept_accuracy(self,pred,acc):
        if (max(pred[0])>acc):
            return np.argmax(pred[0])
        else:
            return len(pred[0])

    def debugMode(self,printlist):
        for count,i in enumerate(printlist):
            print(count,"/",i)

    def responses_generate(self,inp):
        return random.choice(self.response_dictionary[inp])

    def bot_action(self,inp,entities,brain_dict=False):
        if inp=="question_from_user":
            if len(brain_dict)!=0:
                try:
                    return brain_dict[0][1]["define"]
                except:
                    return "Nó khó tả lắm sorry"
            return "Sorry, hiện tại mình chưa biết nữa"

        if inp=="askingme":
            verb_list=self.pos_filter(entities,"verb")
            noun_list=self.pos_filter(entities,"noun")
            if len(brain_dict)!=0:
                return random.choice(["Có","Yess","Yes"])
            else:
                return random.choice(["Không","No","Nah"])
        
        if inp!="askingme" and inp!="question_from_user":
            return self.responses_generate(inp)
  
    def chat(self):
        debugToggle=False
        while True:
            inp=input("You: ")
            # Hotkey
            if inp=="quit":
                break
            if inp=="!debugmode1":
                print("DEV: DEBUG ON")
                debugToggle=True
                continue
            if inp=="!debugmode0":
                print("DEV: DEBUG OFF")
                debugToggle=False
                continue
        
            # Print input shape and type
            # inputs = self.interpreter.get_input_details()
            # print('\n{} output(s):'.format(len(inputs)))
            # for i in range(0, len(inputs)):
            #     print('{} {}'.format(inputs[i]['shape'], inputs[i]['dtype'])) 

            # # Print output shape and type
            # outputs = self.interpreter.get_output_details()
            # print('\n{} output(s):'.format(len(outputs)))
            # for i in range(0, len(outputs)):
            #     print('{} {}'.format(outputs[i]['shape'], outputs[i]['dtype']))   

            intent,pos=self.preprocess(inp)

            self.interpreter.set_tensor(self.input_details[1]["index"],pos)
            self.interpreter.set_tensor(self.input_details[0]["index"],intent)
            self.interpreter.invoke()
            pos_result = self.interpreter.get_tensor(self.output_details[0]['index']) 
            intent_result = self.interpreter.get_tensor(self.output_details[1]['index'])
            # rs=myModel.predict([intent,pos])
            pos_rs=self.posAnalyze(myTokenizer.bi_gram_ize(inp),pos_result)
            brain_dict=self.brain.knowledge_lookup(pos_rs)
            intent_output=self.labellist[self.accept_accuracy(intent_result,0.8)]

            print("You: ",inp)
            if debugToggle==True:
                self.debugMode([
                    ["POS: ",pos_rs],
                    ["Intent: ",intent_output],
                    ["Acc: ",max(intent_result[0])],
                    ["Bot: ",self.bot_action(intent_output,pos_rs,brain_dict)],
                    ["Brain Check: ",brain_dict]
                    ])
            else:
                print("Bot: ",self.bot_action(intent_output,pos_rs,brain_dict))



In [22]:
botBackend=BotBackendLite(mpos,data,bi_gram_list,labellist)
botBackend.chat()

DEV: DEBUG ON
You:  chào cậu
0 / ['POS: ', [['chào', 'verb'], ['cậu', 'noun']]]
1 / ['Intent: ', 'greeting']
2 / ['Acc: ', 1.0]
3 / ['Bot: ', 'chào cậu']
4 / ['Brain Check: ', []]
You:  Hôm nay sao rồi
0 / ['POS: ', [['hôm nay', 'noun'], ['sao', 'part'], ['rồi', 'x']]]
1 / ['Intent: ', '2ndgreeting']
2 / ['Acc: ', 1.0]
3 / ['Bot: ', 'Rất tốt']
4 / ['Brain Check: ', []]
You:  Giúp mình làm hóa được không
0 / ['POS: ', [['giúp', 'verb'], ['mình', 'propn'], ['làm', 'verb'], ['hóa', 'x'], ['được', 'x'], ['không', 'x']]]
1 / ['Intent: ', 'Assist']
2 / ['Acc: ', 1.0]
3 / ['Bot: ', 'Mình biết vấn đề của bạn được không ?']
4 / ['Brain Check: ', []]
You:  cậu giúp mình làm toán với
0 / ['POS: ', [['cậu', 'noun'], ['giúp', 'verb'], ['mình', 'propn'], ['làm', 'verb'], ['toán', 'noun'], ['với', 'adp']]]
1 / ['Intent: ', 'Assist']
2 / ['Acc: ', 1.0]
3 / ['Bot: ', 'Mình luôn ở đây mà']
4 / ['Brain Check: ', [['toán', {'favorite': 'yes', 'info': ['tính toán', 'phương trình', 'đạo hàm'], 'define': 'To